In [6]:
#!/usr/bin/env python3
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.platypus import Image

"""
===============================================================================
DAILY STATEMENT PDF GENERATOR (LOT-BASED POSITIONS + STOP-RISK, FUTURES BACKTEST)
WITH ROLLING VOLATILITY AND MARGIN UTILIZATION VISUALIZATION
===============================================================================

One page per date, showing:
1) Account equity
2) Margin utilization % and $
3) Total notional exposure (gross) and notional/equity
4) Total risk to stops ($ and % of equity)
5) New transactions broken out by BTO / STO / BTC / STC
6) Return on closed trades (realized P&L and % of prior-day equity)
7) Existing portfolio lots (NOT averaged):
   - entry_date, entry_price, stop_loss_price, qty (contracts), multiplier,
     notional, margin_per_contract, margin_used, unrealized_pnl,
     risk_to_stop ($) and risk_to_stop (% equity)

CHARTS: Includes chart pages showing:
  - Two-month rolling annualized volatility of percentage returns
  - Margin utilization rate over time

KEY RULE (per your request):
- Every OPEN transaction (BTO/STO) creates a DISTINCT LOT.
- Lots are NOT averaged together across time.
- Close transactions (BTC/STC) close lots FIFO by default (configurable).

Risk to stop definition (per lot):
- LONG:  max(0, close - stop) * contracts * multiplier * FX
- SHORT: max(0, stop - close) * contracts * multiplier * FX

Inputs expected in BACKTEST_OUTPUT_ROOT:
- portfolio_equity.(csv|parquet)  [required]
- positions.(csv|parquet)         [required]
- trades.csv                      [required]
- instrument_map.csv              [optional; maps trade_symbol -> signal_symbol]

Also reads contract specs (multiplier + margin/contract if available) from:
- 01-futures_universe/futures_contracts_full.parquet

And reads signal close + vol (for stop computation) from:
- 04-futures_price_volatility/.../parquet/<SIGNAL_SYMBOL>.parquet

NEW: Also reads rolling volatility data from:
- 06-reporting/rolling_annualized_volatility.(csv|parquet)
===============================================================================
"""

# ============================================================
# Configuration
# ============================================================

CONFIG = {
    # Backtest output folder (engine outputs)
    "backtest_output_root": Path("./05a-futures_backtest_risk_based/closed"),

    # Contract specs (multiplier + margin columns)
    "contracts_file": Path("./01-futures_universe/futures_contracts_full.parquet"),
    "margin_column_candidates": [
        "initial_margin",
        "init_margin",
        "margin",
        "initial_margin_usd",
        "margin_usd",
    ],

    # Signal data folder (to pull close + ewma32_std for stop calculation)
    "signal_data_dir": Path("./04-futures_price_volatility/norgate_continuous/volatility_measures/parquet"),
    "signal_close_column": "close",
    "signal_vol_column": "ewma32_std",   # DAILY vol

    # FX (single scalar for now)
    "fx": 1.0,

    # Stop-loss model: stop = entry_px * (1 +/- k * sigma_daily)
    "stop_loss_k": 3.0,

    # Lot closing rule
    "close_rule": "FIFO",  # "FIFO" or "LIFO"

    # Output
    "report_dir": Path("./06-reporting"),
    "pdf_name": "daily_statements.pdf",

    # Render options
    "max_tx_rows_per_bucket": 18,
    "max_lot_rows": 22,

    # Chart visualization options
    "include_volatility_chart": True,
    "volatility_chart_file": "rolling_volatility_chart.png",
    "include_margin_chart": True,
    "margin_chart_file": "margin_utilization_chart.png",

    # If contracts file lacks margin, override per trade symbol here:
    "margin_overrides": {
        # "MES": 1500.0
    },
}

# ============================================================
# Utilities
# ============================================================

def fmt_money(x) -> str:
    if x is None or (isinstance(x, float) and not np.isfinite(x)):
        return "N/A"
    return f"{float(x):,.2f}"

def fmt_pct(x) -> str:
    if x is None or (isinstance(x, float) and not np.isfinite(x)):
        return "N/A"
    return f"{100.0*float(x):.2f}%"

def load_df_prefer_parquet(root: Path, stem: str) -> pd.DataFrame:
    pq = root / f"{stem}.parquet"
    csv = root / f"{stem}.csv"
    if pq.exists():
        return pd.read_parquet(pq)
    if csv.exists():
        return pd.read_csv(csv)
    raise FileNotFoundError(f"Missing {stem}.parquet or {stem}.csv in {root}")

def normalize_dates(df: pd.DataFrame, col: str = "date") -> pd.DataFrame:
    df = df.copy()
    df[col] = pd.to_datetime(df[col], errors="coerce")
    return df.dropna(subset=[col]).sort_values(col)

def load_instrument_map(backtest_root: Path) -> dict:
    """trade_symbol -> signal_symbol if instrument_map.csv exists; else {}."""
    p = backtest_root / "instrument_map.csv"
    if not p.exists():
        return {}
    m = pd.read_csv(p)
    m.columns = [str(c).strip().lower() for c in m.columns]
    if "trade_symbol" not in m.columns or "signal_symbol" not in m.columns:
        return {}
    out = {}
    for _, r in m.iterrows():
        ts = str(r["trade_symbol"]).strip().upper()
        ss = str(r["signal_symbol"]).strip().upper()
        if ts and ss and ts.lower() != "nan" and ss.lower() != "nan":
            out[ts] = ss
    return out

def load_contract_specs(contracts_file: Path, margin_candidates: list[str], margin_overrides: dict):
    """Return multiplier_map, margin_map keyed by symbol."""
    if not contracts_file.exists():
        raise FileNotFoundError(f"contracts_file not found: {contracts_file}")

    df = pd.read_parquet(contracts_file)
    df.columns = [str(c).strip().lower() for c in df.columns]

    sym_col_candidates = ["symbol", "ticker", "root", "contract", "instrument"]
    sym_col = next((c for c in sym_col_candidates if c in df.columns), None)
    if sym_col is None:
        raise ValueError(f"Could not find symbol column in contracts file. Columns: {list(df.columns)}")

    pv_col_candidates = ["point_value", "pointvalue", "multiplier", "contract_multiplier"]
    pv_col = next((c for c in pv_col_candidates if c in df.columns), None)
    if pv_col is None:
        raise ValueError(f"Could not find point value column in contracts file. Columns: {list(df.columns)}")

    margin_col = next((c for c in margin_candidates if c in df.columns), None)

    mult_map = {}
    margin_map = {}

    for _, r in df.iterrows():
        sym = str(r[sym_col]).strip().upper()
        if not sym or sym.lower() == "nan":
            continue
        pv = pd.to_numeric(r[pv_col], errors="coerce")
        if pd.notna(pv) and float(pv) > 0:
            mult_map[sym] = float(pv)

        if margin_col is not None:
            mg = pd.to_numeric(r[margin_col], errors="coerce")
            if pd.notna(mg) and float(mg) > 0:
                margin_map[sym] = float(mg)

    for k, v in (margin_overrides or {}).items():
        margin_map[str(k).strip().upper()] = float(v)

    return mult_map, margin_map

def load_signal_series(signal_data_dir: Path, signal_symbol: str, date_index: pd.DatetimeIndex,
                       close_col: str, vol_col: str) -> pd.DataFrame:
    """Return df indexed by date with close and daily vol."""
    p = signal_data_dir / f"{signal_symbol}.parquet"
    if not p.exists():
        raise FileNotFoundError(f"Signal data file not found: {p}")

    df = pd.read_parquet(p)
    df.columns = [str(c).strip().lower() for c in df.columns]
    if "date" not in df.columns:
        if isinstance(df.index, pd.DatetimeIndex):
            df = df.reset_index().rename(columns={"index": "date"})
        else:
            raise ValueError(f"Signal file missing date: {p.name}")

    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"]).sort_values("date").set_index("date")

    if close_col not in df.columns:
        raise ValueError(f"Signal file missing close '{close_col}': {p.name}")
    if vol_col not in df.columns:
        raise ValueError(f"Signal file missing vol '{vol_col}': {p.name}")

    out = df[[close_col, vol_col]].copy()
    return out.reindex(date_index)


def load_summary_statistics(report_dir: Path):
    """
    Load summary statistics from reporting outputs.
    
    Returns:
    --------
    dict with keys: 
        - performance_summary (dict from performance_summary.csv)
        - yearly_returns (DataFrame from yearly_returns.csv)
        - yearly_drawdowns (DataFrame from yearly_drawdowns.csv)
    """
    summary_stats = {}
    
    # Load performance summary (CAGR, overall max DD, etc.)
    try:
        perf_path = report_dir / "performance_summary.csv"
        if perf_path.exists():
            df = pd.read_csv(perf_path)
            summary_stats['performance_summary'] = df.iloc[0].to_dict()
        else:
            summary_stats['performance_summary'] = {}
    except Exception as e:
        print(f"Warning: Could not load performance_summary.csv: {e}")
        summary_stats['performance_summary'] = {}
    
    # Load yearly returns
    try:
        yearly_ret_path = report_dir / "yearly_returns.csv"
        if yearly_ret_path.exists():
            summary_stats['yearly_returns'] = pd.read_csv(yearly_ret_path, index_col=0)
        else:
            summary_stats['yearly_returns'] = pd.DataFrame()
    except Exception as e:
        print(f"Warning: Could not load yearly_returns.csv: {e}")
        summary_stats['yearly_returns'] = pd.DataFrame()
    
    # Load yearly drawdowns
    try:
        yearly_dd_path = report_dir / "yearly_drawdowns.csv"
        if yearly_dd_path.exists():
            summary_stats['yearly_drawdowns'] = pd.read_csv(yearly_dd_path)
        else:
            summary_stats['yearly_drawdowns'] = pd.DataFrame()
    except Exception as e:
        print(f"Warning: Could not load yearly_drawdowns.csv: {e}")
        summary_stats['yearly_drawdowns'] = pd.DataFrame()

    # Load yearly risk metrics
    try:
        yearly_risk_path = report_dir / "yearly_risk_metrics.csv"
        if yearly_risk_path.exists():
            summary_stats['yearly_risk_metrics'] = pd.read_csv(yearly_risk_path)
        else:
            summary_stats['yearly_risk_metrics'] = pd.DataFrame()
    except Exception as e:
        print(f"Warning: Could not load yearly_risk_metrics.csv: {e}")
        summary_stats['yearly_risk_metrics'] = pd.DataFrame()
    
    return summary_stats

# ============================================================
# Rolling Volatility Visualization
# ============================================================

def create_rolling_volatility_chart(report_dir: Path, output_filename: str) -> Path:
    """
    Load rolling annualized volatility data and create a chart.
    
    Parameters:
    -----------
    report_dir : Path to the reporting directory containing rolling_annualized_volatility files
    output_filename : Name of the output PNG file
    
    Returns:
    --------
    Path to the created chart file, or None if data not available
    """
    # Try to load rolling volatility data
    rolling_vol_df = None
    try:
        rolling_vol_df = load_df_prefer_parquet(report_dir, "rolling_annualized_volatility")
    except FileNotFoundError:
        print("Warning: rolling_annualized_volatility data not found. Skipping volatility chart.")
        return None
    
    if rolling_vol_df is None or rolling_vol_df.empty:
        print("Warning: rolling_annualized_volatility data is empty. Skipping volatility chart.")
        return None
    
    # Normalize column names and dates
    rolling_vol_df.columns = [str(c).strip().lower() for c in rolling_vol_df.columns]
    
    if "date" not in rolling_vol_df.columns or "rolling_ann_vol" not in rolling_vol_df.columns:
        print(f"Warning: Expected columns 'date' and 'rolling_ann_vol' not found. Columns: {list(rolling_vol_df.columns)}")
        return None
    
    rolling_vol_df["date"] = pd.to_datetime(rolling_vol_df["date"], errors="coerce")
    rolling_vol_df = rolling_vol_df.dropna(subset=["date", "rolling_ann_vol"]).sort_values("date")
    
    if rolling_vol_df.empty:
        print("Warning: No valid rolling volatility data after cleaning. Skipping volatility chart.")
        return None
    
    # Create the chart
    fig, ax = plt.subplots(figsize=(11, 6))
    
    ax.plot(rolling_vol_df["date"], rolling_vol_df["rolling_ann_vol"], 
            linewidth=1.5, color='#2E86AB', label='Rolling Ann. Volatility')
    
    # Calculate statistics for the chart
    mean_vol = rolling_vol_df["rolling_ann_vol"].mean()
    median_vol = rolling_vol_df["rolling_ann_vol"].median()
    
    # Add horizontal reference lines
    ax.axhline(y=mean_vol, color='#A23B72', linestyle='--', linewidth=1, 
               alpha=0.7, label=f'Mean: {mean_vol:.2%}')
    ax.axhline(y=median_vol, color='#F18F01', linestyle=':', linewidth=1, 
               alpha=0.7, label=f'Median: {median_vol:.2%}')
    
    # Formatting
    ax.set_title('Two-Month Rolling Annualized Volatility of Returns', 
                 fontsize=14, fontweight='bold', pad=15)
    ax.set_xlabel('Date', fontsize=11)
    ax.set_ylabel('Annualized Volatility', fontsize=11)
    
    # Format y-axis as percentage
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    
    # Format x-axis dates
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    plt.xticks(rotation=45, ha='right')
    
    # Add grid
    ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    ax.set_axisbelow(True)
    
    # Add legend
    ax.legend(loc='best', framealpha=0.9)
    
    # Tight layout to prevent label cutoff
    plt.tight_layout()
    
    # Save the chart
    output_path = report_dir / output_filename
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Rolling volatility chart saved to: {output_path}")
    return output_path

# ============================================================
# NEW: Margin Utilization Visualization
# ============================================================

def create_margin_utilization_chart(statements: list[dict], report_dir: Path, output_filename: str) -> Path:
    """
    Create a margin utilization chart from daily statement data.
    
    Parameters:
    -----------
    statements : List of daily statement dicts containing margin_util_pct
    report_dir : Path to the reporting directory
    output_filename : Name of the output PNG file
    
    Returns:
    --------
    Path to the created chart file, or None if data not available
    """
    if not statements:
        print("Warning: No statements provided for margin utilization chart.")
        return None
    
    # Extract margin utilization data
    dates = []
    margin_util_pct = []
    margin_used_dollars = []
    equity_values = []
    
    for st in statements:
        try:
            dt = pd.to_datetime(st['date'])
            mu_pct = st.get('margin_util_pct', np.nan)
            mu_dollars = st.get('margin_used', np.nan)
            eq = st.get('equity', np.nan)
            
            if pd.notna(dt):
                dates.append(dt)
                margin_util_pct.append(mu_pct if np.isfinite(mu_pct) else np.nan)
                margin_used_dollars.append(mu_dollars if np.isfinite(mu_dollars) else np.nan)
                equity_values.append(eq if np.isfinite(eq) else np.nan)
        except Exception:
            continue
    
    if not dates:
        print("Warning: No valid margin data found. Skipping margin utilization chart.")
        return None
    
    # Create DataFrame for easier handling
    margin_df = pd.DataFrame({
        'date': dates,
        'margin_util_pct': margin_util_pct,
        'margin_used': margin_used_dollars,
        'equity': equity_values
    }).sort_values('date')
    
    # Create the chart with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(11, 8), sharex=True)
    
    # ===== Top subplot: Margin Utilization Percentage =====
    ax1.fill_between(margin_df['date'], 0, margin_df['margin_util_pct'], 
                     alpha=0.3, color='#2E86AB')
    ax1.plot(margin_df['date'], margin_df['margin_util_pct'], 
             linewidth=1.5, color='#2E86AB', label='Margin Utilization %')
    
    # Calculate statistics
    mean_util = margin_df['margin_util_pct'].mean()
    max_util = margin_df['margin_util_pct'].max()
    
    # Add horizontal reference lines
    ax1.axhline(y=mean_util, color='#A23B72', linestyle='--', linewidth=1, 
                alpha=0.7, label=f'Mean: {mean_util:.2%}')
    ax1.axhline(y=max_util, color='#E63946', linestyle=':', linewidth=1, 
                alpha=0.7, label=f'Max: {max_util:.2%}')
    
    # Add threshold lines for reference
    ax1.axhline(y=0.25, color='#FFB703', linestyle='-', linewidth=0.8, 
                alpha=0.5, label='25% threshold')
    ax1.axhline(y=0.50, color='#E63946', linestyle='-', linewidth=0.8, 
                alpha=0.5, label='50% threshold')
    
    ax1.set_ylabel('Margin Utilization (%)', fontsize=11)
    ax1.set_title('Margin Utilization Over Time', fontsize=14, fontweight='bold', pad=15)
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    ax1.legend(loc='upper right', framealpha=0.9, fontsize=9)
    ax1.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    ax1.set_axisbelow(True)
    ax1.set_ylim(bottom=0)
    
    # ===== Bottom subplot: Margin Used (Dollars) vs Equity =====
    ax2_twin = ax2.twinx()
    
    # Plot margin used in dollars
    line1, = ax2.plot(margin_df['date'], margin_df['margin_used'], 
                      linewidth=1.5, color='#2E86AB', label='Margin Used ($)')
    ax2.fill_between(margin_df['date'], 0, margin_df['margin_used'], 
                     alpha=0.2, color='#2E86AB')
    
    # Plot equity on secondary axis
    line2, = ax2_twin.plot(margin_df['date'], margin_df['equity'], 
                           linewidth=1.5, color='#4CAF50', label='Account Equity ($)')
    
    ax2.set_xlabel('Date', fontsize=11)
    ax2.set_ylabel('Margin Used ($)', fontsize=11, color='#2E86AB')
    ax2_twin.set_ylabel('Account Equity ($)', fontsize=11, color='#4CAF50')
    
    ax2.tick_params(axis='y', labelcolor='#2E86AB')
    ax2_twin.tick_params(axis='y', labelcolor='#4CAF50')
    
    # Format y-axes with dollar signs and commas
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'${y/1000:.0f}K'))
    ax2_twin.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'${y/1000:.0f}K'))
    
    # Combined legend
    lines = [line1, line2]
    labels = [l.get_label() for l in lines]
    ax2.legend(lines, labels, loc='upper left', framealpha=0.9, fontsize=9)
    
    ax2.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
    ax2.set_axisbelow(True)
    ax2.set_ylim(bottom=0)
    
    # Format x-axis dates
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    plt.xticks(rotation=45, ha='right')
    
    # Tight layout
    plt.tight_layout()
    
    # Save the chart
    output_path = report_dir / output_filename
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Margin utilization chart saved to: {output_path}")
    return output_path

# ============================================================
# Transaction classification (BTO/STO/BTC/STC)
# ============================================================

def split_trade_actions(prev: int, new: int):
    """Return list of (action, qty>0). Handles sign flips."""
    actions = []
    if prev == new:
        return actions
    if prev == 0:
        if new > 0:
            return [("BTO", new)]
        if new < 0:
            return [("STO", abs(new))]
        return []
    if new == 0:
        if prev > 0:
            return [("STC", prev)]
        if prev < 0:
            return [("BTC", abs(prev))]
        return []

    if (prev > 0 and new > 0) or (prev < 0 and new < 0):
        if abs(new) > abs(prev):
            delta = abs(new) - abs(prev)
            return [("BTO", delta)] if new > 0 else [("STO", delta)]
        delta = abs(prev) - abs(new)
        return [("STC", delta)] if prev > 0 else [("BTC", delta)]

    if prev > 0 and new < 0:
        return [("STC", prev), ("STO", abs(new))]
    if prev < 0 and new > 0:
        return [("BTC", abs(prev)), ("BTO", new)]
    return actions

# ============================================================
# Lot accounting
# ============================================================

def compute_stop(entry_px: float, sigma_daily: float, side: str, k: float) -> float:
    if not (np.isfinite(entry_px) and entry_px > 0 and np.isfinite(sigma_daily) and sigma_daily > 0):
        return np.nan
    if side == "LONG":
        return float(entry_px * (1.0 - k * sigma_daily))
    return float(entry_px * (1.0 + k * sigma_daily))

def close_lots(lots: list[dict], qty_to_close: int, close_rule: str):
    """Close qty from lots FIFO/LIFO. Return (remaining_lots, closed_slices)."""
    if qty_to_close <= 0 or not lots:
        return lots, []
    close_rule = close_rule.upper()
    idxs = range(len(lots)) if close_rule == "FIFO" else range(len(lots)-1, -1, -1)

    remaining = [l.copy() for l in lots]
    closed = []

    for i in list(idxs):
        if qty_to_close <= 0:
            break
        lot = remaining[i]
        q = int(lot.get("qty", 0))
        if q <= 0:
            continue
        take = min(q, qty_to_close)
        qty_to_close -= take
        lot["qty"] = q - take
        sl = lot.copy()
        sl["qty_closed"] = int(take)
        closed.append(sl)

    remaining = [l for l in remaining if int(l.get("qty", 0)) > 0]
    return remaining, closed

# ============================================================
# Statement generation
# ============================================================

def build_daily_statements(portfolio: pd.DataFrame,
                           trades: pd.DataFrame,
                           positions: pd.DataFrame,
                           trade_to_signal: dict,
                           mult_map: dict,
                           margin_map: dict,
                           rolling_vol_df: pd.DataFrame = None) -> list[dict]:
    fx = float(CONFIG["fx"])
    k_stop = float(CONFIG["stop_loss_k"])
    close_rule = str(CONFIG["close_rule"]).upper()

    portfolio = portfolio.copy()
    portfolio.columns = [str(c).strip().lower() for c in portfolio.columns]
    portfolio = normalize_dates(portfolio, "date").reset_index(drop=True)

    trades = trades.copy()
    trades.columns = [str(c).strip().lower() for c in trades.columns]
    trades = normalize_dates(trades, "date").reset_index(drop=True)

    positions = positions.copy()
    date_col = next((c for c in positions.columns if str(c).strip().lower() == "date"), None)
    if date_col != "date":
        positions = positions.rename(columns={date_col: "date"})
    positions = normalize_dates(positions, "date").reset_index(drop=True)

    dates = pd.to_datetime(portfolio["date"]).drop_duplicates().sort_values().to_list()
    date_index = pd.DatetimeIndex(dates)

    # Set up rolling volatility lookup
    rolling_vol_lookup = {}
    if rolling_vol_df is not None and not rolling_vol_df.empty:
        vol_df = rolling_vol_df.copy()
        vol_df.columns = [str(c).strip().lower() for c in vol_df.columns]
        if "date" in vol_df.columns and "rolling_ann_vol" in vol_df.columns:
            vol_df["date"] = pd.to_datetime(vol_df["date"], errors="coerce")
            vol_df = vol_df.dropna(subset=["date"]).set_index("date")
            for dt in dates:
                if dt in vol_df.index:
                    rolling_vol_lookup[dt] = float(vol_df.at[dt, "rolling_ann_vol"])

    trade_syms = [c for c in positions.columns if c != "date"]

    signal_cache = {}
    for ts in trade_syms:
        ss = trade_to_signal.get(str(ts).upper(), str(ts).upper())
        if ss in signal_cache:
            continue
        try:
            signal_cache[ss] = load_signal_series(
                CONFIG["signal_data_dir"], ss, date_index,
                CONFIG["signal_close_column"], CONFIG["signal_vol_column"]
            )
        except Exception:
            signal_cache[ss] = pd.DataFrame(index=date_index, data={
                CONFIG["signal_close_column"]: np.nan,
                CONFIG["signal_vol_column"]: np.nan,
            })

    trades_by_date = {d: g for d, g in trades.groupby("date")}
    lots_state = {ts: {"LONG": [], "SHORT": []} for ts in trade_syms}

    def get_close_sigma(trade_symbol: str, dt: pd.Timestamp):
        ss = trade_to_signal.get(str(trade_symbol).upper(), str(trade_symbol).upper())
        ser = signal_cache.get(ss)
        if ser is None or dt not in ser.index:
            return np.nan, np.nan
        c = ser.at[dt, CONFIG["signal_close_column"]]
        s = ser.at[dt, CONFIG["signal_vol_column"]]
        return (float(c) if pd.notna(c) else np.nan), (float(s) if pd.notna(s) else np.nan)

    statements = []

    for i, dt in enumerate(dates):
        dt = pd.Timestamp(dt)

        prow = portfolio.loc[portfolio["date"] == dt]
        if prow.empty:
            continue
        eq = float(prow["equity"].iloc[0])
        pnl = float(prow["pnl"].iloc[0]) if "pnl" in prow.columns and pd.notna(prow["pnl"].iloc[0]) else np.nan
        comm_day = float(prow["commissions"].iloc[0]) if "commissions" in prow.columns and pd.notna(prow["commissions"].iloc[0]) else 0.0

        tx_rows = []
        realized_closed_pnl = 0.0

        todays_trades = trades_by_date.get(dt, None)
        if todays_trades is not None and not todays_trades.empty:
            for _, tr in todays_trades.iterrows():
                if "trade_symbol" in todays_trades.columns:
                    ts = str(tr.get("trade_symbol", "")).strip().upper()
                else:
                    ts = str(tr.get("symbol", "")).strip().upper()
                if not ts or ts.lower() == "nan":
                    continue
                if ts not in lots_state:
                    lots_state[ts] = {"LONG": [], "SHORT": []}

                prev_c = int(tr.get("prev_contracts", 0))
                new_c = int(tr.get("new_contracts", 0))
                actions = split_trade_actions(prev_c, new_c)

                close_px, sigma_daily = get_close_sigma(ts, dt)
                mult = float(mult_map.get(ts, np.nan))
                mgn_ct = float(margin_map.get(ts, np.nan))

                for action, qty in actions:
                    qty = int(qty)
                    if qty <= 0:
                        continue

                    if action in ("BTO", "STO"):
                        side = "LONG" if action == "BTO" else "SHORT"
                        entry_px = close_px
                        stop_px = compute_stop(entry_px, sigma_daily, side, k_stop)

                        lot = {
                            "trade_symbol": ts,
                            "side": side,
                            "qty": qty,
                            "entry_date": dt.date().isoformat(),
                            "entry_price": float(entry_px) if np.isfinite(entry_px) else np.nan,
                            "stop_loss_price": float(stop_px) if np.isfinite(stop_px) else np.nan,
                            "multiplier": mult,
                            "margin_per_contract": mgn_ct,
                            "fx": fx,
                        }
                        lots_state[ts][side].append(lot)

                        tx_rows.append({
                            "date": dt.date().isoformat(),
                            "trade_symbol": ts,
                            "action": action,
                            "qty": qty,
                            "exec_price": entry_px,
                            "stop_loss_price": stop_px,
                            "commission": float(tr.get("commission", 0.0)) if pd.notna(tr.get("commission", np.nan)) else 0.0,
                        })

                    elif action in ("BTC", "STC"):
                        side = "SHORT" if action == "BTC" else "LONG"
                        lots = lots_state[ts][side]
                        remaining, closed_slices = close_lots(lots, qty, close_rule)

                        for sl in closed_slices:
                            q_closed = int(sl.get("qty_closed", 0))
                            entry_px = float(sl.get("entry_price", np.nan))
                            if np.isfinite(entry_px) and np.isfinite(close_px) and np.isfinite(mult):
                                if side == "LONG":
                                    rpnl = q_closed * mult * fx * (close_px - entry_px)
                                else:
                                    rpnl = q_closed * mult * fx * (entry_px - close_px)
                            else:
                                rpnl = np.nan
                            realized_closed_pnl += float(rpnl) if np.isfinite(rpnl) else 0.0

                            tx_rows.append({
                                "date": dt.date().isoformat(),
                                "trade_symbol": ts,
                                "action": action,
                                "qty": q_closed,
                                "exec_price": close_px,
                                "closed_entry_date": sl.get("entry_date"),
                                "closed_entry_price": entry_px,
                                "realized_pnl": float(rpnl) if np.isfinite(rpnl) else np.nan,
                                "commission": float(tr.get("commission", 0.0)) if pd.notna(tr.get("commission", np.nan)) else 0.0,
                            })

                        lots_state[ts][side] = remaining

        open_lots = []
        margin_used = 0.0
        total_notional_gross = 0.0
        total_risk_to_stop = 0.0

        for ts in trade_syms:
            close_px, _ = get_close_sigma(ts, dt)
            mult = float(mult_map.get(ts, np.nan))
            mgn_ct = float(margin_map.get(ts, np.nan))

            for side in ("LONG", "SHORT"):
                for lot in lots_state.get(ts, {}).get(side, []):
                    q = int(lot.get("qty", 0))
                    if q <= 0:
                        continue

                    entry_px = float(lot.get("entry_price", np.nan))
                    stop_px = float(lot.get("stop_loss_price", np.nan))
                    entry_date = lot.get("entry_date", "")

                    notional = abs(q) * mult * fx * close_px if (np.isfinite(close_px) and np.isfinite(mult)) else np.nan
                    mgn_used = abs(q) * mgn_ct if np.isfinite(mgn_ct) else np.nan

                    # Unrealized PnL (mark-to-market)
                    if np.isfinite(close_px) and np.isfinite(entry_px) and np.isfinite(mult):
                        upnl = q * mult * fx * (close_px - entry_px) if side == "LONG" else q * mult * fx * (entry_px - close_px)
                    else:
                        upnl = np.nan

                    # Risk to stop (loss if stopped from current close -> stop level)
                    if np.isfinite(close_px) and np.isfinite(stop_px) and np.isfinite(mult):
                        if side == "LONG":
                            risk_to_stop = max(0.0, (close_px - stop_px)) * abs(q) * mult * fx
                        else:
                            risk_to_stop = max(0.0, (stop_px - close_px)) * abs(q) * mult * fx
                    else:
                        risk_to_stop = np.nan

                    open_lots.append({
                        "trade_symbol": ts,
                        "side": side,
                        "qty": q,
                        "entry_date": entry_date,
                        "entry_price": entry_px,
                        "stop_loss_price": stop_px,
                        "close": close_px,
                        "risk_to_stop": risk_to_stop,
                        "risk_to_stop_pct_equity": (risk_to_stop / eq) if (np.isfinite(risk_to_stop) and eq and eq > 0) else np.nan,
                        "multiplier": mult,
                        "notional": notional,
                        "margin_per_contract": mgn_ct,
                        "margin_used": mgn_used,
                        "unrealized_pnl": upnl,
                    })

                    if np.isfinite(mgn_used):
                        margin_used += float(mgn_used)
                    if np.isfinite(notional):
                        total_notional_gross += float(abs(notional))
                    if np.isfinite(risk_to_stop):
                        total_risk_to_stop += float(risk_to_stop)

        margin_util_pct = (margin_used / eq) if (eq and eq > 0) else np.nan
        notional_leverage = (total_notional_gross / eq) if (eq and eq > 0) else np.nan
        risk_to_stop_pct_equity = (total_risk_to_stop / eq) if (eq and eq > 0) else np.nan

        prev_eq = np.nan
        if i > 0:
            prev_row = portfolio.loc[portfolio["date"] == pd.Timestamp(dates[i-1])]
            if not prev_row.empty:
                prev_eq = float(prev_row["equity"].iloc[0])
        closed_trade_return = (realized_closed_pnl / prev_eq) if (np.isfinite(prev_eq) and prev_eq != 0) else np.nan

        tx_df = pd.DataFrame(tx_rows) if tx_rows else pd.DataFrame(columns=["date","trade_symbol","action","qty","exec_price"])
        tx_buckets = {k: [] for k in ["BTO", "STO", "BTC", "STC"]}
        if not tx_df.empty:
            for k in tx_buckets:
                tx_buckets[k] = tx_df.loc[tx_df["action"] == k].to_dict("records")

        open_lots = sorted(open_lots, key=lambda r: (r["trade_symbol"], r["side"], r["entry_date"], r["entry_price"]))

        # Get rolling volatility for this date
        rolling_ann_vol = rolling_vol_lookup.get(dt, np.nan)

        statements.append({
            "date": dt.date().isoformat(),
            "equity": eq,
            "pnl": pnl,
            "commissions": comm_day,
            "rolling_ann_vol": rolling_ann_vol,
            "margin_used": margin_used,
            "margin_util_pct": margin_util_pct,
            "total_notional_gross": total_notional_gross,
            "notional_leverage": notional_leverage,
            "total_risk_to_stop": total_risk_to_stop,
            "risk_to_stop_pct_equity": risk_to_stop_pct_equity,
            "tx_buckets": tx_buckets,
            "realized_closed_pnl": realized_closed_pnl,
            "closed_trade_return": closed_trade_return,
            "open_lots": open_lots,
        })

    return statements

# ============================================================
# PDF Rendering
# ============================================================

def draw_table(c, left, y, col_specs, rows, max_rows, title=None):
    if title:
        c.setFont("Helvetica-Bold", 10)
        c.drawString(left, y, title)
        y -= 0.16 * inch

    c.setFont("Helvetica-Bold", 8)
    x = left
    for header, _key, w in col_specs:
        c.drawString(x, y, header)
        x += w * inch
    y -= 0.14 * inch
    c.setFont("Helvetica", 8)

    for r in rows[:max_rows]:
        x = left
        for _header, key, w in col_specs:
            val = r.get(key, "")
            if isinstance(val, float):
                if key in ("margin_util_pct", "closed_trade_return", "risk_to_stop_pct_equity"):
                    s = fmt_pct(val)
                elif key in ("qty",):
                    s = str(int(val))
                elif key in ("exec_price","entry_price","stop_loss_price","notional","margin_used","unrealized_pnl","close",
                             "multiplier","margin_per_contract","realized_pnl","closed_entry_price","risk_to_stop"):
                    s = fmt_money(val)
                else:
                    s = f"{val:.4f}"
            else:
                s = str(val)
            c.drawString(x, y, s[:22])
            x += w * inch
        y -= 0.14 * inch
        if y < 0.9 * inch:
            break

    if len(rows) > max_rows:
        c.setFont("Helvetica-Oblique", 8)
        c.drawString(left, y, f"... ({len(rows) - max_rows} more)")
        y -= 0.16 * inch
        c.setFont("Helvetica", 8)

    return y

def draw_summary_statistics_page(c, summary_stats: dict, left: float, top: float):
    """
    Draw a summary statistics page showing CAGR, yearly returns, and drawdowns.
    """
    y = top
    
    # Title
    c.setFont("Helvetica-Bold", 18)
    c.drawString(left, y, "Strategy Performance Summary")
    y -= 0.5 * inch
    
    # Overall Performance Metrics
    c.setFont("Helvetica-Bold", 14)
    c.drawString(left, y, "Overall Performance")
    y -= 0.25 * inch
    
    perf = summary_stats.get('performance_summary', {})
    
    c.setFont("Helvetica", 11)
    
    # Start and end dates
    start_date = perf.get('start_date', 'N/A')
    end_date = perf.get('end_date', 'N/A')
    c.drawString(left, y, f"Period: {start_date} to {end_date}")
    y -= 0.20 * inch
    
    # CAGR
    cagr = perf.get('cagr', np.nan)
    cagr_str = f"{cagr*100:.2f}%" if np.isfinite(cagr) else "N/A"
    c.setFont("Helvetica-Bold", 11)
    c.drawString(left, y, f"CAGR: {cagr_str}")
    c.setFont("Helvetica", 11)
    y -= 0.20 * inch
    
    # Total Return
    total_ret = perf.get('total_return', np.nan)
    total_ret_str = f"{total_ret*100:.2f}%" if np.isfinite(total_ret) else "N/A"
    c.drawString(left, y, f"Total Return: {total_ret_str}")
    y -= 0.20 * inch
    
    # Sharpe Ratio
    sharpe = perf.get('sharpe', np.nan)
    sharpe_str = f"{sharpe:.2f}" if np.isfinite(sharpe) else "N/A"
    c.drawString(left, y, f"Sharpe Ratio: {sharpe_str}")
    y -= 0.20 * inch
    
    # Annual Volatility
    ann_vol = perf.get('ann_vol', np.nan)
    ann_vol_str = f"{ann_vol*100:.2f}%" if np.isfinite(ann_vol) else "N/A"
    c.drawString(left, y, f"Annualized Volatility: {ann_vol_str}")
    y -= 0.20 * inch
    
    # Maximum Drawdown (Overall)
    max_dd = perf.get('max_drawdown', np.nan)
    max_dd_str = f"{max_dd*100:.2f}%" if np.isfinite(max_dd) else "N/A"
    c.setFont("Helvetica-Bold", 11)
    c.drawString(left, y, f"Maximum Drawdown (Overall): {max_dd_str}")
    c.setFont("Helvetica", 11)
    y -= 0.35 * inch
    
    # Yearly Performance Table
    c.setFont("Helvetica-Bold", 14)
    c.drawString(left, y, "Annual Performance by Year")
    y -= 0.25 * inch
    
    yearly_returns = summary_stats.get('yearly_returns', pd.DataFrame())
    yearly_drawdowns = summary_stats.get('yearly_drawdowns', pd.DataFrame())
    
    if not yearly_returns.empty or not yearly_drawdowns.empty:
        # Process yearly returns
        if not yearly_returns.empty:
            yearly_data = yearly_returns.copy()
            yearly_data = yearly_data.reset_index()
            yearly_data.columns = ['year'] + [c.lower() for c in yearly_data.columns[1:]]
        else:
            yearly_data = pd.DataFrame()
        
        # Process yearly drawdowns
        if not yearly_drawdowns.empty:
            dd_data = yearly_drawdowns.copy()
            dd_data.columns = [c.lower() for c in dd_data.columns]
            if not yearly_data.empty:
                yearly_data = yearly_data.merge(dd_data, on='year', how='outer')
            else:
                yearly_data = dd_data
        
        # Process yearly risk metrics
        yearly_risk_metrics = summary_stats.get('yearly_risk_metrics', pd.DataFrame())
        if not yearly_risk_metrics.empty:
            risk_data = yearly_risk_metrics.copy()
            risk_data.columns = [c.lower() for c in risk_data.columns]
            if not yearly_data.empty:
                yearly_data = yearly_data.merge(risk_data, on='year', how='outer')
            else:
                yearly_data = risk_data
                
        # Sort by year
        if 'year' in yearly_data.columns:
            yearly_data = yearly_data.sort_values('year')
        
        # Draw table headers
        c.setFont("Helvetica-Bold", 9)
        x = left
        c.drawString(x, y, "Year")
        x += 0.6 * inch
        c.drawString(x, y, "Return")
        x += 0.9 * inch
        c.drawString(x, y, "Std Dev")
        x += 0.9 * inch
        c.drawString(x, y, "Sharpe")
        x += 0.8 * inch
        c.drawString(x, y, "Max DD")
        x += 0.9 * inch
        c.drawString(x, y, "Calmar")
        y -= 0.18 * inch
        
        # Draw table rows
        c.setFont("Helvetica", 9)
        for _, row in yearly_data.iterrows():
            x = left
            
            # Year
            year = row.get('year', '')
            c.drawString(x, y, str(int(year)) if pd.notna(year) else 'N/A')
            x += 0.6 * inch
            
            # Annual Return
            ann_ret = row.get('yearly_return', np.nan)
            if pd.notna(ann_ret) and np.isfinite(ann_ret):
                ret_str = f"{ann_ret*100:+.1f}%"
            else:
                ret_str = "N/A"
            c.drawString(x, y, ret_str)
            x += 0.9 * inch
            
            # Standard Deviation
            std_dev = row.get('annual_std_dev', np.nan)
            if pd.notna(std_dev) and np.isfinite(std_dev):
                std_str = f"{std_dev*100:.1f}%"
            else:
                std_str = "N/A"
            c.drawString(x, y, std_str)
            x += 0.9 * inch
            
            # Sharpe Ratio
            sharpe = row.get('sharpe_ratio', np.nan)
            if pd.notna(sharpe) and np.isfinite(sharpe):
                sharpe_str = f"{sharpe:.2f}"
            else:
                sharpe_str = "N/A"
            c.drawString(x, y, sharpe_str)
            x += 0.8 * inch
            
            # Max Drawdown
            dd = row.get('max_drawdown', np.nan)
            if pd.notna(dd) and np.isfinite(dd):
                dd_str = f"{dd*100:.1f}%"
            else:
                dd_str = "N/A"
            c.drawString(x, y, dd_str)
            x += 0.9 * inch
            
            # Calmar Ratio
            calmar = row.get('calmar_ratio', np.nan)
            if pd.notna(calmar) and np.isfinite(calmar):
                calmar_str = f"{calmar:.2f}"
            else:
                calmar_str = "N/A"
            c.drawString(x, y, calmar_str)
            
            y -= 0.16 * inch
            
            if y < 1.0 * inch:
                break
    else:
        c.setFont("Helvetica-Oblique", 10)
        c.drawString(left, y, "No yearly performance data available")
        y -= 0.20 * inch
    
    return y

def write_daily_statement_pdf(statements: list[dict], out_pdf: Path, 
                              volatility_chart_path: Path = None,
                              margin_chart_path: Path = None,
                              summary_stats: dict = None):
    c = canvas.Canvas(str(out_pdf), pagesize=letter)
    width, height = letter
    left = 0.65 * inch
    top = height - 0.65 * inch

    # Page 1: Summary statistics
    if summary_stats:
        draw_summary_statistics_page(c, summary_stats, left, top)
        c.showPage()

    # Page 2: Rolling volatility chart
    if volatility_chart_path and volatility_chart_path.exists():
        c.setFont("Helvetica-Bold", 16)
        c.drawString(left, top, "Portfolio Volatility Analysis")
        
        img_width = width - 1.3 * inch
        img_height = img_width * 0.545
        
        y_position = top - 0.5 * inch
        c.drawImage(str(volatility_chart_path), left, y_position - img_height, 
                   width=img_width, height=img_height, preserveAspectRatio=True)
        
        c.setFont("Helvetica", 9)
        note_y = y_position - img_height - 0.3 * inch
        c.drawString(left, note_y, 
                    "Note: Rolling volatility calculated using 42-day (≈2 month) window of daily returns.")
        
        c.showPage()

    # Page 3: Margin utilization chart (NEW)
    if margin_chart_path and margin_chart_path.exists():
        c.setFont("Helvetica-Bold", 16)
        c.drawString(left, top, "Margin Utilization Analysis")
        
        img_width = width - 1.3 * inch
        img_height = img_width * 0.727  # Aspect ratio for 11x8 chart
        
        y_position = top - 0.5 * inch
        c.drawImage(str(margin_chart_path), left, y_position - img_height, 
                   width=img_width, height=img_height, preserveAspectRatio=True)
        
        # Add summary statistics below the chart
        c.setFont("Helvetica", 9)
        note_y = y_position - img_height - 0.3 * inch
        
        # Calculate margin statistics from statements
        if statements:
            margin_pcts = [st.get('margin_util_pct', np.nan) for st in statements]
            margin_pcts = [m for m in margin_pcts if np.isfinite(m)]
            if margin_pcts:
                avg_margin = np.mean(margin_pcts)
                max_margin = np.max(margin_pcts)
                min_margin = np.min(margin_pcts)
                c.drawString(left, note_y, 
                            f"Margin Utilization Summary - Avg: {avg_margin:.2%}, Max: {max_margin:.2%}, Min: {min_margin:.2%}")
                note_y -= 0.15 * inch
        
        c.drawString(left, note_y, 
                    "Note: Low margin utilization indicates significant unused buying power / cash buffer.")
        
        c.showPage()

    # Daily statement pages
    for st in statements:
        y = top

        c.setFont("Helvetica-Bold", 14)
        c.drawString(left, y, f"Daily Statement - {st['date']}")
        y -= 0.30 * inch

        c.setFont("Helvetica", 10)
        c.drawString(left, y, f"Account Equity: ${fmt_money(st['equity'])}")
        y -= 0.18 * inch
        
        # Add rolling volatility display
        rolling_vol = st.get('rolling_ann_vol', np.nan)
        c.drawString(left, y, f"2-Month Rolling Ann. Volatility: {fmt_pct(rolling_vol)}")
        y -= 0.18 * inch
        
        c.drawString(left, y, f"Margin Utilization ($): ${fmt_money(st['margin_used'])}")
        y -= 0.18 * inch
        c.drawString(left, y, f"Margin Utilization (%): {fmt_pct(st['margin_util_pct'])}")
        y -= 0.18 * inch
        c.drawString(left, y, f"Total Notional Exposure (Gross): ${fmt_money(st.get('total_notional_gross', 0.0))}")
        y -= 0.18 * inch
        nl = st.get("notional_leverage", np.nan)
        c.drawString(left, y, f"Notional / Equity: {nl:.2f}x" if np.isfinite(nl) else "Notional / Equity: N/A")
        y -= 0.18 * inch
        c.drawString(left, y, f"Total Risk to Stops ($): ${fmt_money(st.get('total_risk_to_stop', 0.0))}")
        y -= 0.18 * inch
        c.drawString(left, y, f"Risk to Stops (% Equity): {fmt_pct(st.get('risk_to_stop_pct_equity', np.nan))}")
        y -= 0.22 * inch

        c.drawString(left, y, f"Realized P&L (Closings): ${fmt_money(st.get('realized_closed_pnl', 0.0))}")
        y -= 0.18 * inch
        c.drawString(left, y, f"Return on Closed Trades (vs prior equity): {fmt_pct(st.get('closed_trade_return', np.nan))}")
        y -= 0.28 * inch

        c.setFont("Helvetica-Bold", 11)
        c.drawString(left, y, "New Transactions")
        y -= 0.22 * inch

        tx_cols = [
            ("Sym", "trade_symbol", 0.7),
            ("Act", "action", 0.45),
            ("Qty", "qty", 0.45),
            ("Px", "exec_price", 0.8),
            ("EntryDt", "closed_entry_date", 0.8),
            ("EntryPx", "closed_entry_price", 0.8),
            ("R-PnL", "realized_pnl", 0.8),
        ]
        for bucket in ["BTO", "STO", "BTC", "STC"]:
            rows = st["tx_buckets"].get(bucket, [])
            if rows:
                y = draw_table(c, left, y, tx_cols, rows, CONFIG["max_tx_rows_per_bucket"], title=bucket)
                y -= 0.10 * inch

        if y < 2.2 * inch:
            c.showPage()
            y = top

        c.setFont("Helvetica-Bold", 11)
        c.drawString(left, y, "Existing Portfolio (Lots)")
        y -= 0.22 * inch

        lot_cols = [
            ("Sym", "trade_symbol", 0.55),
            ("Side", "side", 0.4),
            ("Qty", "qty", 0.32),
            ("EntryDt", "entry_date", 0.70),
            ("EntryPx", "entry_price", 0.65),
            ("Stop", "stop_loss_price", 0.65),
            ("Close", "close", 0.55),
            ("Risk$", "risk_to_stop", 0.65),
            ("Risk%", "risk_to_stop_pct_equity", 0.55),
            ("Mult", "multiplier", 0.55),
            ("Notional", "notional", 0.75),
            ("Mgn/ct", "margin_per_contract", 0.55),
            ("Mgn$", "margin_used", 0.55),
            ("uPnL", "unrealized_pnl", 0.55),
        ]

        open_lots = st.get("open_lots", [])
        y = draw_table(c, left, y, lot_cols, open_lots, CONFIG["max_lot_rows"], title=None)

        c.showPage()

    c.save()

# ============================================================
# Main
# ============================================================

def main():
    root = Path(CONFIG["backtest_output_root"])
    if not root.exists():
        raise FileNotFoundError(f"Backtest output root not found: {root}")

    portfolio = load_df_prefer_parquet(root, "portfolio_equity")
    positions = load_df_prefer_parquet(root, "positions")
    trades_path = root / "trades.csv"
    if not trades_path.exists():
        raise FileNotFoundError(f"trades.csv not found in {root}")
    trades = pd.read_csv(trades_path)

    trade_to_signal = load_instrument_map(root)
    mult_map, margin_map = load_contract_specs(
        CONFIG["contracts_file"],
        CONFIG["margin_column_candidates"],
        CONFIG.get("margin_overrides", {}),
    )

    # Load rolling volatility data
    out_dir = Path(CONFIG["report_dir"])
    rolling_vol_df = None
    try:
        rolling_vol_df = load_df_prefer_parquet(out_dir, "rolling_annualized_volatility")
    except FileNotFoundError:
        print("Warning: rolling_annualized_volatility data not found. Daily statements will not include rolling volatility.")

    statements = build_daily_statements(
        portfolio=portfolio,
        trades=trades,
        positions=positions,
        trade_to_signal=trade_to_signal,
        mult_map=mult_map,
        margin_map=margin_map,
        rolling_vol_df=rolling_vol_df,
    )

    out_dir.mkdir(parents=True, exist_ok=True)
    
    # Load summary statistics
    summary_stats = load_summary_statistics(out_dir)
    
    # Create rolling volatility chart if enabled
    volatility_chart_path = None
    if CONFIG.get("include_volatility_chart", True):
        volatility_chart_path = create_rolling_volatility_chart(
            out_dir, 
            CONFIG.get("volatility_chart_file", "rolling_volatility_chart.png")
        )
    
    # NEW: Create margin utilization chart if enabled
    margin_chart_path = None
    if CONFIG.get("include_margin_chart", True):
        margin_chart_path = create_margin_utilization_chart(
            statements,
            out_dir, 
            CONFIG.get("margin_chart_file", "margin_utilization_chart.png")
        )
    
    out_pdf = out_dir / CONFIG["pdf_name"]
    write_daily_statement_pdf(statements, out_pdf, volatility_chart_path, margin_chart_path, summary_stats)

    print(f"\nDaily statements PDF written to: {out_pdf.resolve()}")
    num_chart_pages = sum([
        1 if summary_stats else 0,
        1 if volatility_chart_path else 0,
        1 if margin_chart_path else 0,
    ])
    print(f"Pages: {num_chart_pages} summary/chart + {len(statements)} daily statements = {num_chart_pages + len(statements)} total")
    print(f"Charts included:")
    print(f"  - Summary statistics page: {'Yes' if summary_stats else 'No'}")
    print(f"  - Rolling volatility chart: {'Yes' if volatility_chart_path else 'No'}")
    print(f"  - Margin utilization chart: {'Yes' if margin_chart_path else 'No'}")

if __name__ == "__main__":
    main()

Rolling volatility chart saved to: 06-reporting\rolling_volatility_chart.png
Margin utilization chart saved to: 06-reporting\margin_utilization_chart.png

Daily statements PDF written to: C:\TWS API\source\pythonclient\TradingIdeas\Futures\06-reporting\daily_statements.pdf
Pages: 3 summary/chart + 462 daily statements = 465 total
Charts included:
  - Summary statistics page: Yes
  - Rolling volatility chart: Yes
  - Margin utilization chart: Yes
